## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [4]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [5]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [6]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [7]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [8]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [9]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [10]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)
print(sentences_ted[:2])

[['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new'], ['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']]


Two sample processed sentences:

In [11]:
len(sentences_ted)

266694

In [12]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [13]:
vectorizer = CountVectorizer(min_df=3)
a = vectorizer.fit_transform([' '.join(sent) for sent in sentences_ted]).toarray()
print(a.shape)
#print(a)
#print(vectorizer.vocabulary_)
names = vectorizer.get_feature_names()
#print(names)
count_words = np.sum(a, axis=0)
print(len(count_words))

(266694, 28372)
28372


'for i in np.argsort(count_words)[:50]:\n    counts_ted_top1000.append(i)\nprint(counts_ted_top1000)'

In [36]:
indexes_ted_top1000 = np.argsort(count_words)[-1000:]
counts_ted_top1000 = np.sort(count_words)[-1000:]

words_top_ted = []
for i in indexes_ted_top1000:
    words_top_ted.append(names[i])

print(words_top_ted)

['wife', 'village', 'mine', 'fire', 'decision', 'expect', 'taught', '80', 'teachers', 'meaning', 'older', 'finding', 'shown', 'buildings', 'somewhere', 'goal', 'cases', 'screen', 'connected', 'tree', 'fight', 'police', 'families', 'interest', 'faster', 'supposed', 'allowed', 'models', 'projects', 'leaders', 'skin', 'usually', 'african', 'conversation', 'standing', 'extraordinary', 'role', 'including', 'objects', 'named', 'student', 'doctor', 'vision', 'continue', 'further', 'son', 'happiness', 'object', 'door', 'essentially', 'strong', 'fall', 'safe', 'positive', 'poverty', 'east', 'plan', 'jobs', 'grew', 'materials', 'exciting', 'results', 'lead', '500', 'choose', 'worse', 'creative', 'road', 'organization', 'watching', 'biggest', 'plant', 'follow', 'trust', 'anyway', 'medicine', 'individuals', 'extremely', 'wait', 'table', 'movie', 'wants', 'speed', 'schools', 'moved', 'response', '11', 'zero', 'favorite', 'everywhere', 'plants', 'memory', 'add', 'break', 'onto', 'dream', 'mass', 'de

Plot distribution of top-1000 words

In [56]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")

show(p)

### Part 3: Train Word2Vec

In [16]:
from gensim.models import Word2Vec

In [17]:
#model_ted = Word2Vec(sentences_ted, size=100, window=5, min_count=5, workers=4)

In [18]:
#model_ted.save('model_ted')

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [19]:
model_ted = Word2Vec.load('model_ted')
model_ted.most_similar("man")

[('woman', 0.8294707536697388),
 ('guy', 0.8152415752410889),
 ('girl', 0.744369626045227),
 ('gentleman', 0.7435609102249146),
 ('lady', 0.7348114252090454),
 ('boy', 0.7328171730041504),
 ('soldier', 0.7239211797714233),
 ('kid', 0.6812607049942017),
 ('poet', 0.663948118686676),
 ('john', 0.6615406274795532)]

In [20]:
model_ted.most_similar("computer")

[('machine', 0.7279675006866455),
 ('software', 0.7263143658638),
 ('device', 0.7001446485519409),
 ('robot', 0.69496750831604),
 ('3d', 0.6794931888580322),
 ('camera', 0.6778303384780884),
 ('printer', 0.6639512181282043),
 ('laser', 0.6441822052001953),
 ('interface', 0.6379550695419312),
 ('simulation', 0.6342030167579651)]

In [57]:
model_ted.most_similar("language")

[('english', 0.6559274792671204),
 ('mathematics', 0.6485158801078796),
 ('culture', 0.6480445861816406),
 ('narrative', 0.6135907173156738),
 ('discipline', 0.600331723690033),
 ('beauty', 0.5931773781776428),
 ('technique', 0.5863878726959229),
 ('emotion', 0.585470974445343),
 ('logic', 0.5851237773895264),
 ('desire', 0.5849875211715698)]

In [ ]:
model_ted.wv.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])

In [63]:
print(model_ted.predict_output_word(['I','like']))

[('looks', 0.9924283), ('feels', 0.0052567362), ('looked', 0.00045622015), ('d', 0.00039840906), ('look', 0.00024918301), ('sounds', 0.00019324874), ('things', 0.00013808795), ('places', 7.2217284e-05), ('felt', 4.1551219e-05), ('smells', 1.9273108e-05)]


#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [37]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

In [38]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [39]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [69]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [75]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [76]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [77]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [73]:
sentences_wiki_tokenized = []
for sent_str in sentences_wiki:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_wiki_tokenized.append(tokens)
print(sentences_wiki_tokenized[:2])

[['inch', 'mm', 'guns'], ['microsoft', 's', 'entertainment', 'and', 'devices', 'division', 'experienced', 'a', 'huge', 'increase', 'in', 'revenue', 'largely', 'driven', 'by', 'the', 'release', 'of', 'halo', 'and', 'posted', 'a', 'quarterly', 'profit', 'for', 'the', 'first', 'time', 'in', 'two', 'years']]


In [48]:
vectorizer_1 = CountVectorizer(min_df=3)
a = vectorizer_1.fit_transform([' '.join(sent) for sent in sentences_wiki_tokenized]).toarray()
print(a.shape)
names_wiki = vectorizer_1.get_feature_names()
count_words_wiki = np.sum(a, axis=0)
print(len(count_words_wiki))

(34136, 16412)
16412


In [49]:
indexes_wiki_top1000 = np.argsort(count_words_wiki)[-1000:]
counts_wiki_top1000 = np.sort(count_words_wiki)[-1000:]

words_top_wiki = []
for i in indexes_wiki_top1000:
    words_top_wiki.append(names_wiki[i])

print(words_top_wiki)

['particular', 'latter', 'refused', 'minor', 'woman', 'direction', 'via', 'theme', 'empire', 'contains', 'remains', 'increase', 'producer', 'mass', 'lyrics', 'edge', 'minute', 'names', 'need', 'era', 'always', 'health', 'artist', 'americans', 'wind', 'winter', 'greatest', 'hospital', 'earned', 'dance', 'angeles', 'intended', 'language', 'la', 'actor', 'defeated', 'saint', 'federal', 'takes', 'course', 'atlantic', 'rd', 'hour', 'smaller', 'done', 'schools', 'key', 'african', 'edward', 'lord', 'pop', 'department', 'corps', 'richard', 'independent', 'tracks', 'things', 'deal', 'festival', 'cut', 'structure', 'books', 'immediately', 'debut', 'think', 'your', 'double', 'offered', 'martin', 'islands', 'performances', 'report', 'goals', 'soviet', 'museum', 'fight', 'railway', 'ran', 'includes', 'sales', 'male', 'meeting', 'marriage', 'terms', 'stars', 'removed', 'shows', 'cost', 'appear', 'blue', 'suggested', 'percent', 'singer', 'yet', 'subsequently', 'native', 'moving', 'defense', 'listed',

In [50]:
hist, edges = np.histogram(counts_wiki_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")

show(p)

In [51]:
#model_wiki = Word2Vec(sentences_wiki_tokenized, size=100, window=5, min_count=5, workers=4)
#model_wiki.save('model_wiki')

In [52]:
model_wiki = Word2Vec.load('model_wiki')
model_wiki.most_similar("man")

[('intimate', 0.9800007343292236),
 ('job', 0.9799379706382751),
 ('child', 0.9787692427635193),
 ('interview', 0.9738097190856934),
 ('script', 0.9694739580154419),
 ('kind', 0.9687577486038208),
 ('mistake', 0.9684147238731384),
 ('voice', 0.9675799012184143),
 ('looked', 0.9675579071044922),
 ('instrument', 0.9658246040344238)]

In [53]:
model_wiki.most_similar("computer")

[('flood', 0.9952467679977417),
 ('allegations', 0.9934673309326172),
 ('dawn', 0.993462324142456),
 ('conservatives', 0.9921740293502808),
 ('tie', 0.9918414354324341),
 ('gomez', 0.9917268753051758),
 ('sale', 0.9916478395462036),
 ('phillips', 0.9916264414787292),
 ('violence', 0.9916070103645325),
 ('shooting', 0.9915851950645447)]

#### t-SNE visualization

In [54]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [55]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)